In [ ]:
%tensorflow_version 1.x
!apt-get install ffmpeg freeglut3-dev xvfb
!pip install stable-baselines[mpi]==2.10.0

!git clone -l -s git://github.com/wjwswain/chaRL.git chaRL
%cd chaRL/games/blackjack
from env import GameEnv

import gym
import numpy as np
from stable_baselines.deepq.policies import MlpPolicy
from stable_baselines import A2C

TensorFlow 1.x selected.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
freeglut3-dev is already the newest version (2.8.1-3).
freeglut3-dev set to manually installed.
ffmpeg is already the newest version (7:3.4.6-0ubuntu0.18.04.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  xvfb
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 784 kB of archives.
After this operation, 2,266 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 xvfb amd64 2:1.19.6-1ubuntu4.4 [784 kB]
Fetched 784 kB in 1s (586 kB/s)
Selecting previously unselected package xvfb.
(Reading database ... 144465 files and directories currently installed.)
Preparing to unpack .../xvfb_2%3a1.19.6-1ubuntu4.4_amd64.deb ...
Unpacking xvfb (2:1.19.6-1ubuntu4.4) ..

In [ ]:
from stable_baselines import DQN

env = GameEnv(5000,"Wesley")
model = DQN(MlpPolicy, env, verbose=1)
model.learn(total_timesteps=25000)
model.save("dqn_mlp_blackjack")

while True:
    action, _states = model.predict(obs)
    obs, rewards, dones, info = env.step(action)
    env.render()

In [1]:
!git clone -l -s git://github.com/wjwswain/chaRL.git chaRL
%cd chaRL/games/blackjack
from env import GameEnv

env = GameEnv(5000,"Wesley")
env.reset()
for _ in range(10):
    env.render()
    env.step(env.action_space.sample())
env.close()

Cloning into 'chaRL'...
remote: Enumerating objects: 105, done.
remote: Counting objects: 100% (105/105), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 105 (delta 26), reused 94 (delta 18), pack-reused 0
Receiving objects: 100% (105/105), 1.06 MiB | 8.13 MiB/s, done.
Resolving deltas: 100% (26/26), done.
/content/chaRL/games/blackjack
Example: 5000
Round Reward: 0
Example bet 365:
<bound method Hand.display of <env.Hand object at 0x7f830ae6cd68>>

Dealer:
Dealer
??
None

Example: 4635
Round Reward: -365
Example bet 355:
<bound method Hand.display of <env.Hand object at 0x7f830ae36fd0>>

Dealer:
Dealer
??
None

Example: 4280
Round Reward: -355
Example bet 100:
<bound method Hand.display of <env.Hand object at 0x7f830ae369b0>>

Dealer:
Dealer
??
None

Example: 4180
Round Reward: -100
Example bet 115:
<bound method Hand.display of <env.Hand object at 0x7f830a196cc0>>

Dealer:
Dealer
??
None

Example bet 115:
<bound method Hand.display of <env.Hand object at 0x7f830a